# Predict  drugs' protein interaction profiles or proteins' molecular interaction profiles with Komet and LCIdb

1. Downloading utils from Github
2. Downloading dataset from Zenodo
3. Predict proteins' molecular interaction profiles with Komet and LCIdb
4. Predict  drugs' protein interaction profiles  with Komet and LCIdb

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torch.optim as optim

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import zipfile
import pickle

import time

from sklearn.metrics import  average_precision_score,  roc_curve, confusion_matrix, precision_score, recall_score, auc

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device_cpu = torch.device("cpu")
device_cpu = device
print( device )

mytype = torch.float16 # to save memory (only on GPU)
mytype = torch.float32

cpu


In [2]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 25.9 MB/s eta 0:00:00


### 1. Download the data from a GitHub repo.

In [2]:
!wget -q https://github.com/Guichaoua/komet/raw/main/komet/komet.py

zsh:1: command not found: wget


In [3]:
import komet

cpu


### 2. Download the data from Zenodo.

You can directly download train_arr.pkl or make your training dataset.

In [5]:
LCIdb_path = '/Users/gguichaoua/Dropbox/gwenn/these/2024_01_Komet_JCIM/Zenodo_v2/LCIdb_v2.csv'
 
LCIdb = pd.read_csv(LCIdb_path,low_memory=False)
LCIdb.head()

,smiles,fasta,ChEMBL ID,PubChem ID,IUPHAR ID,Ligand names,Target,uniprot,mean,mean pIC50,...,mean pKd,min,min pIC50,min pKi,min pKd,max,max pIC50,max pKi,max pKd,score
0,BrC(=C\c1ccccc1)/C=N/n1cnnc1,MNPTLILAAFCLGIASATLTFDHSLEAQWTKWKAMHNRLYGMNEEG...,CHEMBL3190095,6861939.0,NaN,sid24812872,ctsl,P07711,4.2,4.2,...,NaN,4.2,4.2,NaN,NaN,4.2,4.2,NaN,NaN,0.5
1,BrC(Cn1ncc2c(N3CCCC3)ncnc21)c1ccccc1,MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADG...,CHEMBL382012,11617657.0,NaN,1-(2-bromo-2-phenylethyl)-4-(pyrrolidin-1-yl)-...,src,P12931,5.0,NaN,...,NaN,5.0,NaN,5.0,NaN,5.0,NaN,5.0,NaN,0.5
2,BrC(Cn1ncc2c(N3CCCCC3)ncnc21)c1ccccc1,MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADG...,CHEMBL203922,11696703.0,NaN,1-(2-bromo-2-phenylethyl)-4-(piperidin-1-yl)-1...,src,P12931,5.8,NaN,...,NaN,5.8,NaN,5.8,NaN,5.8,NaN,5.8,NaN,0.5
3,BrC(Cn1ncc2c(NCc3ccccc3)ncnc21)c1ccccc1,MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADG...,CHEMBL382153,11545984.0,NaN,n-benzyl-1-(2-bromo-2-phenylethyl)-1h-pyrazolo...,src,P12931,5.5,NaN,...,NaN,5.5,NaN,5.5,NaN,5.5,NaN,5.5,NaN,0.5
4,BrC1=CC2CNCCC2S1,MSGADRSPNAGAAPDSAPGQAAVASAYQRFEPRAYLRNNYAPPRGD...,CHEMBL398475,44447166.0,NaN,"2-bromo-3a,4,5,6,7,7a-hexahydrothieno[3,2-c]py...",pnmt,P11086,5.9,NaN,...,NaN,5.9,NaN,5.9,NaN,5.9,NaN,5.9,NaN,0.5


In [6]:
train = komet.make_trains_full('/Users/gguichaoua/Dropbox/gwenn/these/2024_01_Komet_JCIM/Zenodo_v2/LCIdb_v2.csv','/Users/gguichaoua/Dropbox/gwenn/these/2024_01_Komet_JCIM/Zenodo_v2/',nb_trains=5)

nombre de smiles:  271180
nombre de fasta:  2060


/Users/gguichaoua/Dropbox/gwenn/these/chemogenomique/komet/docs/source/vignettes/komet.py:2619: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(indsmiles_index_with_nan,0)
/Users/gguichaoua/Dropbox/gwenn/these/chemogenomique/komet/docs/source/vignettes/komet.py:2622: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(indfasta_index_with_nan,0)


matrice d'interactions:  (2060, 271180)
train (793596, 3)
train (793596, 3)
train (793596, 3)
train (793596, 3)
train (793596, 3)


In [7]:
train_0 = train[0]
train_0.head()

,SMILES,Target Sequence,Label
0,Brc1cccc(-n2cnc(-c3ccccc3Nc3ccncc3)n2)c1,MQSKVLLAVALWLCVETRAASVGLPSVSLDLPRLSIQKDILTIKAN...,1
1,Brc1cccc(-n2cnc(-c3ccccc3Nc3ccncc3)n2)c1,MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...,0
2,Brc1cccc(-n2cnc(-c3ccccc3Nc3ccncc3)n2)c1,MVSYWDTGVLLCALLSCLLLTGSSSGSKLKDPELSLKGTQHIMQAG...,1
3,Brc1cccc(Nc2ncc(-c3ccccc3)s2)n1,MQSKVLLAVALWLCVETRAASVGLPSVSLDLPRLSIQKDILTIKAN...,1
4,Brc1cccc(Oc2ccc3c(/C=C/c4ccccc4)n[nH]c3c2)c1,MQSKVLLAVALWLCVETRAASVGLPSVSLDLPRLSIQKDILTIKAN...,1


### 3. Predict proteins' molecular interaction profiles with Komet and LCIdb

You need to download to Zenodo and save in ./data, the 3 files dict_ind2fasta_all.data, dict_ind2fasta_all_K_prot.data and fasta_uniprot_family.csv

In [72]:
name_drug = 'aspirin'
smiles_drug = 'CC1=CC(=O)C2=C(C1=O)C=CC=C2'
df_pred_drug,pred = komet.predict_drug_profile(train,smiles_drug,mM = 3000,dM = 1000,lamb = 1e-6)
df_pred_drug.head()

The drug is not in the dataset
kernel prot shape torch.Size([2060, 2060])
protein features shape torch.Size([2060, 2060])
mol kernel shape torch.Size([3000, 271181])
L-BFGS time: 622.5025 seconds
mol kernel shape torch.Size([3000, 271181])
L-BFGS time: 624.5909 seconds
mol kernel shape torch.Size([3000, 271181])
L-BFGS time: 620.6984 seconds
mol kernel shape torch.Size([3000, 271181])
L-BFGS time: 631.1737 seconds
mol kernel shape torch.Size([3000, 271181])
L-BFGS time: 637.6766 seconds


,Target,uniprot,family,Label_known,Proba_predicted_mean,Proba_predicted_std,Proba_predicted_min,Proba_predicted_max
0,ces1,P23141,type-B carboxylesterase/lipase family,-1,0.999757,0.000130,0.999517,0.999869
1,ces2,O00748,type-B carboxylesterase/lipase family,-1,0.996522,0.002068,0.992532,0.998258
2,maoa,P21397,flavin monoamine oxidase family,-1,0.994963,0.003355,0.988908,0.998240
3,maob,P27338,flavin monoamine oxidase family,-1,0.991253,0.004065,0.983757,0.996147
4,prkch,C0HM02,Not known,-1,0.983708,0.010395,0.966951,0.995497


#### Remarks : VERIFIER CHEMBL25
In Drugbank, for aspirin, we can see 16 targets but only 2 bind to aspirin (ptgs1 and ptgs2). For others, the pharmacological action is unknown. Our prediction finds these 2 connections and rejects the others.

In [83]:
df_pred_drug[df_pred_drug['uniprot'].isin(['P23219','P35354'])]

,Target,uniprot,family,Label_known,Proba_predicted_mean,Proba_predicted_std,Proba_predicted_min,Proba_predicted_max
12,ptgs2,P35354,prostaglandin G/H synthase family,-1,0.902107,0.020121,0.884362,0.936080
24,ptgs1,P23219,prostaglandin G/H synthase family,-1,0.710457,0.095295,0.590268,0.854953


In [84]:
df_pred_drug[df_pred_drug['uniprot'].isin(['P23219','P35354','Q04828','Q13131','P54646','Q9Y478','O43741','P54619','Q9UGJ0','Q9UGI9','P25101','P04637','P11021','P51812',  'P25963', 'P98066' ,'P29466', 'P42574', 'O14920','P28482','Q8TD08','P27361','P31152','Q16659','Q13164','P24385','P01106','P12004','Q99519'])]

,Target,uniprot,family,Label_known,Proba_predicted_mean,Proba_predicted_std,Proba_predicted_min,Proba_predicted_max
12,ptgs2,P35354,prostaglandin G/H synthase family,-1,0.902107,0.020121,0.884362,0.936080
24,ptgs1,P23219,prostaglandin G/H synthase family,-1,0.710457,0.095295,0.590268,0.854953
163,akr1c1,Q04828,aldo/keto reductase family,-1,0.120188,0.101206,0.014647,0.294680
219,casp3,P42574,peptidase family,-1,0.066869,0.045929,0.018455,0.142134
779,ednra,P25101,G-protein coupled receptor 1 family,-1,0.005633,0.006959,0.000550,0.019016
838,mapk1,P28482,protein kinase superfamily,-1,0.004710,0.008011,0.000240,0.020719
849,neu1,Q99519,glycosyl hydrolase 33 family,-1,0.004558,0.000592,0.003846,0.005532
884,casp1,P29466,peptidase family,-1,0.004312,0.001560,0.002841,0.006690
1006,myc,P01106,Not known,-1,0.003082,0.000511,0.002452,0.003981
1033,ccnd1,P24385,cyclin family,-1,0.002879,0.004249,0.000073,0.011333


### 4. Predict  drugs' protein interaction profiles  with Komet and LCIdb

You need to download to Zenodo and save in ./data, the 3 files dict_ind2fasta_all.data, dict_ind2fasta_all_K_prot.data and fasta_uniprot_family.csv

In [132]:
df_family = pd.read_csv("data/fasta_uniprot_family.csv")

In [133]:
name_protein = 'ptgs1'
fasta_protein = df_family[df_family["Target"] == "ptgs1"]["fasta"].values[0]
df_pred_protein = komet.predict_protein_profile(train,fasta_protein,mM = 3000,dM = 1000,lamb = 1e-6)
df_pred_protein.head()

kernel prot shape torch.Size([2060, 2060])
protein features shape torch.Size([2060, 2060])
mol kernel shape torch.Size([3000, 271180])
L-BFGS time: 598.0121 seconds
mol kernel shape torch.Size([3000, 271180])
L-BFGS time: 589.7209 seconds
mol kernel shape torch.Size([3000, 271180])
L-BFGS time: 607.9472 seconds
mol kernel shape torch.Size([3000, 271180])
L-BFGS time: 591.4509 seconds
mol kernel shape torch.Size([3000, 271180])
L-BFGS time: 618.3177 seconds


,SMILES,Label,Proba_predicted_mean,Proba_predicted_std,Proba_predicted_min,Proba_predicted_max
250550,COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3)cc2-c2ccc(Cl)...,-1.0,0.999925,0.000031,0.999883,0.999976
6791,CC(C)(C)c1cc(C(=O)c2ccccc2)cc2c1OCC2(C)C,-1.0,0.999882,0.000064,0.999802,0.999956
250552,COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3OC)cc2-c2ccc(C...,-1.0,0.999855,0.000060,0.999773,0.999951
250542,COc1ccc(-n2nc(CC(C(=O)O)c3ccc(Cl)cc3)cc2-c2ccc...,-1.0,0.999781,0.000079,0.999669,0.999913
250555,COc1ccc(C(Cc2cc(-c3ccc(Cl)cc3)n(-c3ccc(OC)cc3)...,-1.0,0.999781,0.000079,0.999669,0.999913


In [140]:
df_pred_protein.head(15)

,index,SMILES,Label,Proba_predicted_mean,Proba_predicted_std,Proba_predicted_min,Proba_predicted_max
0,250550,COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3)cc2-c2ccc(Cl)...,-1.0,0.999925,0.000031,0.999883,0.999976
1,6791,CC(C)(C)c1cc(C(=O)c2ccccc2)cc2c1OCC2(C)C,-1.0,0.999882,0.000064,0.999802,0.999956
2,250552,COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3OC)cc2-c2ccc(C...,-1.0,0.999855,0.000060,0.999773,0.999951
3,250542,COc1ccc(-n2nc(CC(C(=O)O)c3ccc(Cl)cc3)cc2-c2ccc...,-1.0,0.999781,0.000079,0.999669,0.999913
4,250555,COc1ccc(C(Cc2cc(-c3ccc(Cl)cc3)n(-c3ccc(OC)cc3)...,-1.0,0.999781,0.000079,0.999669,0.999913
5,8619,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,-1.0,0.999779,0.000118,0.999595,0.999910
6,250549,COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3)cc2-c2ccc(C)c...,-1.0,0.999701,0.000266,0.999175,0.999877
7,250540,COc1ccc(-n2nc(CC(C(=O)O)c3ccc(C)cc3)cc2-c2ccc(...,-1.0,0.999654,0.000119,0.999457,0.999832
8,250541,COc1ccc(-n2nc(CC(C(=O)O)c3ccc(Cl)cc3)cc2-c2ccc...,-1.0,0.999654,0.000119,0.999457,0.999832
9,250544,COc1ccc(-n2nc(CC(C(=O)O)c3cccc(C)c3)cc2-c2ccc(...,-1.0,0.999624,0.000180,0.999328,0.999848


In [141]:
# the protein is in the dataset : 1 (DTI+), 0 (DTI-), -1 (unknown DTI)
df_pred_protein.value_counts('Label')

Label
-1.0    271032
 0.0        74
 1.0        74
dtype: int64

In [143]:
for i in range(15):
    print("rank",i+1,df_pred_protein['SMILES'].values[i],df_pred_protein['Proba_predicted_mean'].values[i])


rank 1 COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3)cc2-c2ccc(Cl)cc2)cc1 0.9999254941940308
rank 2 CC(C)(C)c1cc(C(=O)c2ccccc2)cc2c1OCC2(C)C 0.999882173538208
rank 3 COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3OC)cc2-c2ccc(Cl)cc2)cc1 0.9998553276062012
rank 4 COc1ccc(-n2nc(CC(C(=O)O)c3ccc(Cl)cc3)cc2-c2ccc(Cl)cc2)cc1 0.9997808814048768
rank 5 COc1ccc(C(Cc2cc(-c3ccc(Cl)cc3)n(-c3ccc(OC)cc3)n2)C(=O)O)cc1 0.9997808814048768
rank 6 COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1 0.9997786998748779
rank 7 COc1ccc(-n2nc(CC(C(=O)O)c3ccccc3)cc2-c2ccc(C)cc2)cc1 0.9997011065483093
rank 8 COc1ccc(-n2nc(CC(C(=O)O)c3ccc(C)cc3)cc2-c2ccc(Cl)cc2)cc1 0.999653947353363
rank 9 COc1ccc(-n2nc(CC(C(=O)O)c3ccc(Cl)cc3)cc2-c2ccc(C)cc2)cc1 0.999653947353363
rank 10 COc1ccc(-n2nc(CC(C(=O)O)c3cccc(C)c3)cc2-c2ccc(Cl)cc2)cc1 0.9996240496635437
rank 11 COc1ccc(-n2nc(CC(C(=O)O)c3cccc(OC)c3)cc2-c2ccc(Cl)cc2)cc1 0.9996052384376526
rank 12 Cc1c(CC(=O)O)c2ccccc2n1C(=O)c1ccc(Cl)cc1 0.9994504809379577
rank 13 COc1ccc2c(c1)c(CC(=O)Oc1cccnc1)c(C)n2C(=O

### Remarks : 
For the predictions : 
* For the second drug CHEMBL275120, the binding was found: https://pubchem.ncbi.nlm.nih.gov/compound/10852345#section=Biological-Test-Results
* For the 6th drug (Indomethacin), the binding was found:  https://pubchem.ncbi.nlm.nih.gov/compound/3715#section=BioAssay-Results
* For the 12th durg (CHEMBL1241992), the binding was known